# Preprocessing Sberbank data

* Import data
* Split in training and validation sets
* Handle missing values
* Handle ordinal variables
* Handle nominal variables
* Scaling continuous variables
* Handle outliers by trimming?

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.model_selection import train_test_split

In [2]:
# Import data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
macro = pd.read_csv("data/macro.csv")

In [3]:
# Quick peek
train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


## Split

In [4]:
# Split in training and validation sets
X, y = train.iloc[:, :291], train.iloc[:, 291]
X_train, X_val, y_train, y_val = \
    train_test_split(X, y, test_size = 0.3, random_state = 0)

print(y_train.shape)
print(y_val.shape)
print(X_train.shape)
print(X_val.shape)

(21329,)
(9142,)
(21329, 291)
(9142, 291)


In [5]:
X_train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
22671,22674,2014-09-19,34,19.0,16.0,17.0,1.0,1977.0,1.0,9.0,...,87,33,3,1,8,32,2,8,81,12
9852,9855,2013-08-15,159,95.0,16.0,NaN,NaN,NaN,NaN,NaN,...,546,313,127,22,132,196,1,88,167,12
9490,9493,2013-07-29,85,47.0,8.0,17.0,1.0,2008.0,3.0,11.0,...,51,21,4,1,10,13,0,1,63,6
16430,16433,2014-03-24,43,28.0,3.0,5.0,2.0,1960.0,2.0,5.0,...,29,11,3,0,8,25,0,4,65,12
21120,21123,2014-07-28,43,31.0,3.0,5.0,1.0,1961.0,2.0,6.0,...,20,5,2,0,7,13,0,1,47,6


## Missing values

In [45]:
# Percent missing
p_missing = X_train.isnull().sum() / X_train.shape[0]
print(p_missing[p_missing != 0])

# Problematic: build_year, state, hospital_beds_raion, 
# cafe_sum_500_min_price_avg, cafe_sum_500_max_price_avg, cafe_avg_price_500

# Potentially problematic: max_floor, material, num_room, kitch_sq

floor                                    0.005157
material                                 0.314736
num_room                                 0.314736
state                                    0.446153
preschool_quota                          0.218716
school_quota                             0.218576
raion_build_count_with_material_info     0.163346
build_count_block                        0.163346
build_count_wood                         0.163346
build_count_frame                        0.163346
build_count_brick                        0.163346
build_count_monolith                     0.163346
build_count_panel                        0.163346
build_count_foam                         0.163346
build_count_slag                         0.163346
build_count_mix                          0.163346
raion_build_count_with_builddate_info    0.163346
build_count_before_1920                  0.163346
build_count_1921-1945                    0.163346
build_count_1946-1970                    0.163346


In [7]:
# Look at some of these variables
X_train[["build_year", "state", "hospital_beds_raion", "max_floor", "material", "num_room", "kitch_sq"]].head()

,build_year,state,hospital_beds_raion,max_floor,material,num_room,kitch_sq
22671,1977.0,2.0,2672.0,17.0,1.0,1.0,9.0
9852,NaN,NaN,1786.0,NaN,NaN,NaN,NaN
9490,2008.0,3.0,NaN,17.0,1.0,3.0,11.0
16430,1960.0,3.0,1183.0,5.0,2.0,2.0,5.0
21120,1961.0,2.0,4087.0,5.0,1.0,2.0,6.0


In [14]:
# Ordinal features could be treated as nominal, and then missing can be a category of it's own:
# state, material, num_room for example
print(X_train["state"].unique())
print(X_train["material"].unique())
print(X_train["num_room"].unique())



[  2.  nan   3.   1.   4.]
[  1.  nan   2.   5.   6.   4.   3.]
[  1.  nan   3.   2.   4.   0.   5.   6.   8.  17.  19.   9.]


In [46]:
# Function for one-hot encoding for nominal variables
def one_hot(df, cols):
    from sklearn.preprocessing import OneHotEncoder
    
    ohe = OneHotEncoder(categorical_features = ["state", "material", "num_room"])
    print(ohe.fit_transform(df).toarray().head())
    

In [54]:
# Perform one-hot encoding
#one_hot(X_train, 10)
np.where(X_train.columns == "state")

(array([10]),)

In [25]:
# Median imputation?
print(X_train["max_floor"].median())
print(X_train["life_sq"].median())
print(X_train["kitch_sq"].median())
print(X_train["hospital_beds_raion"].median())
print(X_train["build_year"].median())

12.0
30.0
6.0
990.0
1979.0


In [43]:
# Function for median imputation

def median_impute(df, cols):
    from sklearn.preprocessing import Imputer
    imr = Imputer(missing_values = "NaN", strategy = "median", axis = 0)
    imr = imr.fit(df[cols])
    df_imputed = imr.transform(df[cols].values)
    return(df_imputed)

In [44]:
cols = ["max_floor", "life_sq", "kitch_sq", "hospital_beds_raion", "build_year"]
print(X_train[cols].head())


# Impute
X_train.loc[:, cols] = median_impute(X_train, cols)
print(X_train[cols].head())

       max_floor  life_sq  kitch_sq  hospital_beds_raion  build_year
22671       17.0     19.0       9.0               2672.0      1977.0
9852        12.0     95.0       6.0               1786.0      1979.0
9490        17.0     47.0      11.0                990.0      2008.0
16430        5.0     28.0       5.0               1183.0      1960.0
21120        5.0     31.0       6.0               4087.0      1961.0
       max_floor  life_sq  kitch_sq  hospital_beds_raion  build_year
22671       17.0     19.0       9.0               2672.0      1977.0
9852        12.0     95.0       6.0               1786.0      1979.0
9490        17.0     47.0      11.0                990.0      2008.0
16430        5.0     28.0       5.0               1183.0      1960.0
21120        5.0     31.0       6.0               4087.0      1961.0


/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
